# Amazon Bedrock AgentCore Runtime での MCP サーバーのホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime で MCP（Model Context Protocol）サーバーをホストする方法を学びます。Amazon Bedrock AgentCore Python SDK を使用して、MCP ツールを Amazon Bedrock AgentCore と互換性のある MCP サーバーとしてラップします。

Amazon Bedrock AgentCore Python SDK は MCP サーバーの実装の詳細を処理するため、ツールのコア機能に集中できます。このSDKはあなたのコードを AgentCore 標準化された MCP プロトコル契約に変換し、直接通信を可能にします。

### チュートリアルの詳細

| 情報               | 詳細                                                   |
|:------------------|:--------------------------------------------------------|
| チュートリアルタイプ | ツールのホスティング                                     |
| ツールタイプ       | MCP サーバー                                            |
| チュートリアルコンポーネント | AgentCore Runtime での MCP サーバーのホスティング   |
| チュートリアル分野   | 分野横断的                                             |
| 例の複雑さ         | 簡単                                                   |
| 使用 SDK          | Amazon BedrockAgentCore Python SDK と MCP              |

### チュートリアルのアーキテクチャ

このチュートリアルでは、MCP サーバーを AgentCore ランタイムにデプロイする方法について説明します。

デモンストレーションのために、3 つのツール（`add_numbers`、`multiply_numbers`、`greet_user`）を持つシンプルな MCP サーバーを使用します。

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="40%"/>
</div>

### チュートリアルの主な機能

* カスタムツールを使用した MCP サーバーの作成
* MCP サーバーのローカルでのテスト
* Amazon Bedrock AgentCore Runtime での MCP サーバーのホスティング
* 認証を使用したデプロイ済み MCP サーバーの呼び出し

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* 設定済みの AWS 認証情報
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) ライブラリ
* 実行中の Docker

In [ ]:
#!uv add -r requirements.txt --active

In [1]:
import os
os.environ['AWS_PROFILE'] = 'cline2'

# べき等ではないため完全に一から再実行する場合は `rm .bedrock_agentcore.yaml` を実行してください。

## MCP（Model Context Protocol）の理解

MCPは、AI モデルが外部データやツールに安全にアクセスできるようにするプロトコルです。主要な概念：

* **ツール**：AI がアクションを実行するために呼び出せる関数
* **Streamable HTTP**：AgentCore Runtime で使用される転送プロトコル
* **セッション分離**：各クライアントは `Mcp-Session-Id` ヘッダーを介して分離されたセッションを取得
* **ステートレス操作**：サーバーはスケーラビリティのためにステートレス操作をサポートする必要がある

AgentCore Runtime は、MCP サーバーがデフォルトパスとして `0.0.0.0:8000/mcp` でホストされることを想定しています。

### プロジェクト構造

適切な構造でプロジェクトをセットアップしましょう：

```
mcp_server_project/
├── mcp_server.py              # メイン MCP サーバーコード
├── my_mcp_client.py          # ローカルテスト用クライアント
├── my_mcp_client_remote.py   # リモートテスト用クライアント
├── requirements.txt          # 依存関係
└── __init__.py              # Python パッケージマーカー
```

## MCP サーバーの作成

ここでは、3 つのシンプルなツールを使って MCP サーバーを作成しましょう。このサーバーは AgentCore Runtime との互換性に必要な `stateless_http=True` を指定した FastMCP を使用します。

In [2]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"こんにちは, {name}! はじめまして."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

Overwriting mcp_server.py


### このコードの機能

* **FastMCP**: あなたのツールをホストできる MCP サーバーを作成します
* **@mcp.tool()**: Python 関数を MCP ツールに変換するデコレーター
* **stateless_http=True**: AgentCore Runtime との互換性に必要
* **ツール**: 異なるタイプの操作を示す 3 つのシンプルなツール

## ローカルテスト用クライアントの作成

AgentCore Runtime にデプロイする前に、MCP サーバーをローカルでテストするためのクライアントを作成しましょう：

In [3]:
%%writefile my_mcp_client.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:18000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

Overwriting my_mcp_client.py


### ローカルでのテスト

MCP サーバーをローカルでテストするには：

1. **ターミナル 1**： MCP サーバーを起動する
   ```bash
   FASTMCP_PORT=18000 uv run mcp_server.py
   ```
   
2. **ターミナル 2**： テストクライアントを実行する
   ```bash
   uv run my_mcp_client.py
   ```

出力に 3 つのツールが表示されるはずです。

## Amazon Cognito による認証の設定

AgentCore Runtime には認証が必要です。デプロイした MCP サーバーへのアクセスのために JWT トークンを提供する Amazon Cognito を使用します。

この関数は、Amazon Cognito User Pool を作成し、アプリケーションクライアント、テストユーザーを設定して、認証トークンを取得するまでの一連のプロセスを自動化しています。主に MCP サーバーの認証に使用するための設定を行っています。

1. Boto3 セッションの初期化(cognito-idp のクライアントを初期化)
2. User Pool の作成(MCPServerPool という名前の User Pool を作成)
3. アプリケーションクライアントの作成(User Pool に MCPServerPoolClient という名前のクライアントを追加)
4. テストユーザーの作成(testuser というユーザー名でテストユーザーを作成)
5. 永続的なパスワードの設定(MyPassword123!)
6. ユーザー認証とトークン取得(作成したユーザーで認証を実行し、認証結果から Bearer Token を取得)

セル実行のたびに新規で上記が作成されます。

In [4]:
import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '.')
utils_dir = os.path.abspath(utils_dir)

sys.path.insert(0, utils_dir)

from utils import create_agentcore_role, setup_cognito_user_pool

In [5]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_1VKOs98iH
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_1VKOs98iH/.well-known/openid-configuration
Client ID: 5caa5p8ihkrc3m0h09814ph1n
Bearer Token: eyJraWQiOiJIajZPemgxSFlNMGtOcDNsQmtZQmZYRmhuZWpFSTVHTU5RTll2RmREUnVjPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiIzNGY4NjRlOC01MDMxLTcwZWItNGZjZi1kZGY0ODQ3NThmODAiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV8xVktPczk4aUgiLCJjbGllbnRfaWQiOiI1Y2FhNXA4aWhrcmMzbTBoMDk4MTRwaDFuIiwib3JpZ2luX2p0aSI6Ijc2NTc1YjIwLWNmNzItNGYzMC1iMjU3LWUxZTE1NGE4ZTU4MCIsImV2ZW50X2lkIjoiMTU1ODc0ZTQtNGYzOS00ZDAxLWE4YzItY2MwOTc2ZTA0MWUyIiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJhd3MuY29nbml0by5zaWduaW4udXNlci5hZG1pbiIsImF1dGhfdGltZSI6MTc1MzYwMzUwNiwiZXhwIjoxNzUzNjA3MTA2LCJpYXQiOjE3NTM2MDM1MDYsImp0aSI6Ijg2ZTIyZWEzLWZkYzQtNDNlMy1iNWJiLWFmOWViYTlkY2E0MiIsInVzZXJuYW1lIjoidGVzdHVzZXIifQ.oT_JlM12cl96mk9jXdxguRoPY3p2LfoeQw882WTExhpc8TgcnBQ6bb2NmsA8FHfByWdNz58l-sWJps

## ステップ 5: IAM 実行ロールの作成

始める前に、AgentCore Runtime 用の IAM ロールを作成しましょう。このロールは、ランタイムが動作するために必要な権限を提供します。

In [6]:
tool_name = "mcp_server"
print(f"Creating IAM role for {tool_name}...")
agentcore_iam_role = create_agentcore_role(agent_name=tool_name)
print(f"IAM role created ✓")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

Creating IAM role for mcp_server...
使用中のプロファイル: cline2
使用中のリージョン: us-east-1
アカウントID: 320462930492
IAMロール agentcore-mcp_server-role の作成を試みます
ロール agentcore-mcp_server-role は既に存在します - 削除して再作成します
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': 'aab2edb7-40c4-4909-9089-a95ec8c4fdc9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 27 Jul 2025 08:05:09 GMT', 'x-amzn-requestid': 'aab2edb7-40c4-4909-9089-a95ec8c4fdc9', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
agentcore-mcp_server-role を削除中
agentcore-mcp_server-role を再作成中
attaching role policy agentcore-mcp_server-role
IAM role created ✓
Role ARN: arn:aws:iam::320462930492:role/agentcore-mcp_server-role


## AgentCore ランタイムデプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および要件ファイルを含む AgentCore ランタイムデプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップでは、アプリケーションコードに基づいて Docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [7]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

Entrypoint parsed: file=/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/02-hosting-MCP-server/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: mcp_server


Using AWS region: us-east-1
All required files found ✓
Configuring AgentCore Runtime...


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/02-hosting-MCP-server/Dockerfile
Generated .dockerignore: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/02-hosting-MCP-server/.dockerignore
Keeping 'mcp_server' as default agent
Bedrock AgentCore configured: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/02-hosting-MCP-server/.bedrock_agentcore.yaml


Configuration completed ✓


## MCP サーバーを AgentCore ランタイムに起動する

Docker ファイルができたので、MCP サーバーを AgentCore ランタイムに起動しましょう。これにより Amazon ECR リポジトリと AgentCore ランタイムが作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [1]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

Launching MCP server to AgentCore Runtime...
This may take several minutes...


NameError: name 'agentcore_runtime' is not defined

## AgentCore ランタイムのステータス確認

AgentCore ランタイムをデプロイしたので、そのデプロイ状況を確認し、準備が整うまで待ちましょう：

In [9]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

Checking AgentCore Runtime status...


Retrieved Bedrock AgentCore status for: mcp_server


Initial status: READY
✓ AgentCore Runtime is READY!
Final status: READY


## リモートアクセスのための設定の保存

デプロイされた MCP サーバーを呼び出す前に、Agent ARN と Cognito 設定を AWS Systems Manager Parameter Store および AWS Secrets Manager に保存して、簡単に取得できるようにしましょう：

In [10]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Cognito credentials updated in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:320462930492:runtime/mcp_server-88vFBQ4ff8


## リモートテスト用クライアントの作成

次に、デプロイした MCP サーバーをテストするためのクライアントを作成しましょう。このクライアントは AWS から必要な認証情報を取得し、デプロイされたサーバーに接続します：

In [1]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client_remote.py


## MCP サーバーのデプロイをテストする

リモートクライアントを使用して、デプロイした MCP サーバーをテストしましょう：

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

## MCP ツールをリモートで呼び出す

ここでは、ツールを一覧表示するだけでなく、MCP の完全な機能を実証するためにそれらを呼び出す拡張クライアントを作成しましょう：

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## テスト ツールの呼び出し

MCP ツールを実際に呼び出して、テストしてみましょう：

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## 次のステップ

AgentCore Runtime に MCP サーバーを正常にデプロイしたので、次のことができます：

1. **ツールの追加**: MCP サーバーに追加のツールを拡張する
2. **カスタム認証**: カスタム JWT 認証機能を実装する
3. **統合**: 他の AgentCore サービスと統合する

## クリーンアップ（任意）

このチュートリアルで作成したリソースをクリーンアップしたい場合は、以下のセルを実行してください：

In [ ]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
iam_client = boto3.client('iam')
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    print("Deleting IAM role policies...")
    policies = iam_client.list_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
    )

    for policy_name in policies['PolicyNames']:
        iam_client.delete_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyName=policy_name
        )
    
    iam_client.delete_role(
        RoleName=agentcore_iam_role['Role']['RoleName']
    )
    print("✓ IAM role deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")

    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

# 🎉 おめでとうございます！

以下のことに成功しました：

✅ カスタムツールを備えた **MCP サーバーの作成**  
✅ MCP クライアントを使用した **ローカルでのテスト**  
✅ Amazon Cognito による **認証の設定**  
✅ AgentCore Runtime を使用した **AWS へのデプロイ**  
✅ 適切な認証による **リモートからの呼び出し**  
✅ **MCP の概念** とベストプラクティスの学習  

あなたの MCP サーバーは現在 Amazon Bedrock AgentCore Runtime 上で実行されており、本番環境での使用準備が整いました！

## 概要

このチュートリアルでは、以下の方法を学びました：
- FastMCP を使用した MCP サーバーの構築
- AgentCore との互換性のためのステートレス HTTP トランスポートの設定
- Amazon Cognito による JWT 認証の設定
- AWS 上での MCP サーバーのデプロイと管理
- ローカルとリモートの両方でのテスト
- ツール呼び出しのための MCP クライアントの使用

デプロイされた MCP サーバーは、より大規模な AI アプリケーションやワークフローに統合できるようになりました！